In [1]:
from imap_tools import MailBox, AND
import os
import pandas as pd
import re

In [2]:
# username = "adm.happyenergy@gmail.com"
# password = "mxtl agjv dcsn tcnv" #adm.happyenergy@gmail.com
# _from = "engenhariahappysolar@gmail.com"

username = "engenhariahappysolar@gmail.com"
password = "jtpg hdvr pmat rtap" #engenhariahappysolar@gmail.com
_from = 'noreplyportalgd@neoenergia.com'

query = "Portal da Geração Distribuída: Solicitação "


mailbox = MailBox('imap.gmail.com').login(username, password)
lista_emails = mailbox.fetch(AND(from_=_from))
lista_emails_com_query = [email for email in lista_emails if query.lower() in email.subject.lower()]
print("Emails encontrados:", len(lista_emails_com_query))

Emails encontrados: 450


In [ ]:
Clientes = []
for email in lista_emails_com_query:

    match_num = re.search(r'Solicitação (\d+)', email.subject)
    numero_solicitacao = match_num.group(1) if match_num else "Não encontrado"

    match_paragrafo = re.search(r'<p>Prezado\(a\) (.*?),', email.html, re.IGNORECASE | re.DOTALL)
    Nome_cliente = match_paragrafo.group(1).strip() if match_paragrafo else "Não encontrado"
    if Nome_cliente not in [c['nome_cliente'] for c in Clientes]:
        Clientes.append({
        'numero_solicitacao': numero_solicitacao,
        'nome_cliente': Nome_cliente,
        'data': email.date
    })
    # print(Nome_cliente)
    # break
len(Clientes)
df = pd.DataFrame(Clientes)
df.to_csv('clientes_all.csv', index=False)
print("Lista gerada e salva em clientes_all.csv")

In [ ]:
# for email in lista_emails:
#     print(email.subject)
#     print(email.date)

# Modificação para extrair número da solicitação e nome do cliente
dados_clientes = []

for email in lista_emails_com_query:
    # Extrair número da solicitação do assunto (assumindo formato "Solicitação XXXX")
    match_num = re.search(r'Solicitação (\d+)', email.subject)
    numero_solicitacao = match_num.group(1) if match_num else "Não encontrado"
    
    # Extrair nome do cliente do corpo do email (assumindo que está em uma linha como "Cliente: Nome")
    # Você pode ajustar o regex conforme o formato exato do email
    match_nome = re.search(r'Prezado\(a\)\s+(.+?),', email.html, re.IGNORECASE)
    nome_cliente = match_nome.group(1).strip() if match_nome else "Não encontrado"  
    if numero_solicitacao not in dados_clientes['numero_solicitacao']:
        dados_clientes.append({
            'numero_solicitacao': numero_solicitacao,
            'nome_cliente': nome_cliente,
            'data': email.date
        })
    # print(f"Extraído - Solicitação: {numero_solicitacao}, Cliente: {nome_cliente}")
    print(email.text)

# Criar DataFrame e salvar em CSV
df = pd.DataFrame(dados_clientes)
df.to_csv('clientes_ativos.csv', index=False)
print("Lista gerada e salva em clientes_ativos.csv")
print(df)

In [ ]:
BASE_DIR = os.getcwd()
CSV_FILES = [
    os.path.join(BASE_DIR, 'Solcitações/PortalGD_minhas_solicitacoes.csv'),
    os.path.join(BASE_DIR, 'Solcitações/PortalGD_minhas_solicitacoes (1).csv'),
]
def load_active_codes():
    codes = set()
    for path in CSV_FILES:
        if os.path.exists(path):
            try:
                df = pd.read_csv(path, encoding='latin1')
            except Exception:
                df = pd.read_csv(path, encoding='utf-8', errors='ignore')
            if df.shape[1] >= 2:
                vals = df.iloc[:, 1].astype(str).str.strip()
                codes.update(vals.tolist())
    return codes


In [ ]:
cont = 0
for csv_file in CSV_FILES:
    df_temp = pd.read_csv(csv_file, encoding='latin1')
    if cont == 0:
        df = df_temp
        cont += 1
    else:
        df = pd.concat([df, df_temp], ignore_index=True)
# print(df.shape())